In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import seaborn as sns
import matplotlib.pyplot as plt



# 1. Load the datasets

k20 = pd.read_csv( "kaggle2020.csv" )
k20 = pd.DataFrame(k20)
k21 = pd.read_csv( "kaggle2021.csv" )
k21 = pd.DataFrame(k21)
k22 = pd.read_csv( "kaggle2022.csv" )
k22 = pd.DataFrame(k22)

/var/folders/k2/fbh1klm51sd2zvn3kx2sqcr00000gn/T/ipykernel_10545/1336181873.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  k20 = pd.read_csv( "kaggle2020.csv" )
/var/folders/k2/fbh1klm51sd2zvn3kx2sqcr00000gn/T/ipykernel_10545/1336181873.py:13: DtypeWarning: Columns (0,195,201,285,286,287,288,289,290,291,292) have mixed types. Specify dtype option on import or set low_memory=False.
  k21 = pd.read_csv( "kaggle2021.csv" )
/var/folders/k2/fbh1klm51sd2zvn3kx2sqcr00000gn/T/ipykernel_10545/1336181873.py:15: DtypeWarning: Columns (0,208,225,255,257,260,270,271,277) have mixed types. Specify dtype option on import or set low_memory=False.
  k22 = pd.read_csv( "kaggle2022.csv" )


In [2]:

# 2. Drop participants whose job roles are students, unemployed, or not stated
k20_filtered = k20[~k20['Q5'].isin([None, '', 'Student', 'Unemployed'])]
k21_filtered = k21[~k21['Q5'].isin([None, '', 'Student', 'Unemployed'])]
k22_filtered = k22[~k22['Q23'].isin([None, '', 'Student', 'Unemployed'])]


# 3. Prepare and aggregate the data
# Extracting relevant columns for analysis, including the uniform 'JobRole' column
questions = {
    'income': {'2020': 'Q24', '2021': 'Q25', '2022': 'Q29'},
    'gender': {'2020': 'Q2', '2021': 'Q2', '2022': 'Q3'},
    'age': {'2020': 'Q1', '2021': 'Q1', '2022': 'Q2'},  # Note the change for 2022
    'education level': {'2020': 'Q4', '2021': 'Q4', '2022': 'Q8'},
    'job role': {'2020': 'Q5', '2021': 'Q5', '2022': 'Q23'},
    'years of programming experience': {'2020': 'Q6', '2021': 'Q6', '2022': 'Q11'},  # Note the change for 2022
    'Spending on ML/Cloud Services': {'2020': 'Q25', '2021': 'Q26', '2022': 'Q30'},
    'Employer Incorporating ML Methods': {'2020': 'Q22', '2021': 'Q23', '2022': 'Q27'},
    'Country of Residence': {'2020': 'Q3', '2021': 'Q3', '2022': 'Q4'}
}

# Combining the datasets
combined_data = pd.DataFrame()
for year, df in [('2020', k20_filtered), ('2021', k21_filtered), ('2022', k22_filtered)]:
    relevant_cols = {questions[q][year]: q for q in questions}
    selected_data = df[relevant_cols.keys()].rename(columns=relevant_cols)
    selected_data['Year'] = year
    combined_data = combined_data.append(selected_data)

# Convert all relevant columns to categorical type
for col in questions:
    combined_data[col] = combined_data[col].astype('category')

combined_data




/var/folders/k2/fbh1klm51sd2zvn3kx2sqcr00000gn/T/ipykernel_10545/3421815612.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(selected_data)
/var/folders/k2/fbh1klm51sd2zvn3kx2sqcr00000gn/T/ipykernel_10545/3421815612.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(selected_data)
/var/folders/k2/fbh1klm51sd2zvn3kx2sqcr00000gn/T/ipykernel_10545/3421815612.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(selected_data)


,income,gender,age,education level,job role,years of programming experience,Spending on ML/Cloud Services,Employer Incorporating ML Methods,Country of Residence,Year
0,What is your current yearly compensation (appr...,What is your gender? - Selected Choice,What is your age (# years)?,What is the highest level of formal education ...,Select the title most similar to your current ...,For how many years have you been writing code ...,Approximately how much money have you (or your...,Does your current employer incorporate machine...,In which country do you currently reside?,2020
2,"100,000-124,999",Man,30-34,Master’s degree,Data Engineer,5-10 years,"$100,000 or more ($USD)","We have well established ML methods (i.e., mod...",United States of America,2020
3,"15,000-19,999",Man,35-39,Bachelor’s degree,Software Engineer,10-20 years,$0 ($USD),No (we do not use ML methods),Argentina,2020
4,"125,000-149,999",Man,30-34,Master’s degree,Data Scientist,5-10 years,"$10,000-$99,999","We have well established ML methods (i.e., mod...",United States of America,2020
5,NaN,Man,30-34,Master’s degree,Software Engineer,3-5 years,NaN,NaN,Japan,2020
...,...,...,...,...,...,...,...,...,...,...
23993,NaN,Man,22-24,Master’s degree,NaN,3-5 years,NaN,NaN,United States of America,2022
23994,NaN,Man,60-69,Bachelor’s degree,NaN,20+ years,NaN,NaN,United States of America,2022
23995,"25,000-29,999",Man,25-29,Master’s degree,Engineer (non-software),< 1 years,$100-$999,No (we do not use ML methods),Turkey,2022
23996,"15,000-19,999",Woman,35-39,Doctoral degree,Research Scientist,3-5 years,$0 ($USD),"We recently started using ML methods (i.e., mo...",Israel,2022


In [3]:
# 4. Conduct and print the correlation analysis using Cramer's V
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

for col in questions.keys():
    v = cramers_v(combined_data[col], combined_data['job role'])
    print(f"The Cramer's V for {col} is {v:.3f}")


The Cramer's V for income is 0.220
The Cramer's V for gender is 0.450
The Cramer's V for age is 0.319
The Cramer's V for education level is 0.391
The Cramer's V for job role is 1.000
The Cramer's V for years of programming experience is 0.385
The Cramer's V for Spending on ML/Cloud Services is 0.388
The Cramer's V for Employer Incorporating ML Methods is 0.436
The Cramer's V for Country of Residence is 0.219
